In [0]:
# Para a quinta visualização é preciso duas tabelas.
# Uma para as conexões e outra para os nós.

# O sexto gráfico é gerado com os mesmo dados das conexões. 

In [0]:
import pandas as pd
import numpy as np

In [3]:
alunos_caxias = pd.read_csv('alunos_caxias_2018.csv')
alunos_caxias.head()

,ID_ALUNO,NU_ANO_CENSO,CO_IES,TP_GRAU_ACADEMICO,TP_MODALIDADE_ENSINO,NO_CURSO,NU_IDADE,ANO_INGRESSO,TP_SITUACAO,TP_NIVEL_ACADEMICO,TP_SEXO
0,71CCAB7D5BAE6603B3995CA2EB8F9534,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,34,2014,Cursando,Graduação,Masculino
1,FBC7BF40B3F9AC1C46F78CFF70AC141B,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,28,2018,Cursando,Graduação,Masculino
2,43ABA22A4C7DC2F6E69C71CE90D339A0,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,27,2017,Matrícula trancada,Graduação,Masculino
3,10707CB293B32AD1BC23EB641765BCE7,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,28,2014,Matrícula trancada,Graduação,Feminino
4,7C95F1E0E536524DC3281C4452030F72,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,54,2007,Matrícula trancada,Graduação,Feminino


In [4]:
# selecionar os registros com a situação 'Cursando' e que começaram na IES em 2018
situacao = alunos_caxias.TP_SITUACAO == 'Cursando' # situação cursando
nivel_academico = (alunos_caxias.TP_NIVEL_ACADEMICO == 'Graduação') #nível de graduação presencial e a distância 
ano = (alunos_caxias.ANO_INGRESSO == 2018)
cursando = alunos_caxias.loc[situacao & nivel_academico & ano,:].copy() # alunos cursando em 2018
cursando.CO_IES.value_counts() # alunos que se matricularam e estão cursando em 2018 em cada IES

Centro Universitário Da Serra Gaúcha                       4370
Universidade De Caxias Do Sul                              3001
Centro Universitário Uniftec                               1859
Faculdade Anhanguera De Caxias Do Sul                       410
Faculdade De Tecnologia Da Serra Gaúcha - Caxias Do Sul     111
Faculdade Murialdo                                          100
Faculdade Nossa Senhora De Fátima                           100
Faculdade Ideau De Caxias Do Sul                             78
Name: CO_IES, dtype: int64

In [5]:
# códigos que não estão vinculados a algum curso em 2018 (desvinculado do curso ou matrícula trancada)
anos = (alunos_caxias.ANO_INGRESSO.isin([2016,2017,2018])) # só interessa registros recentes
situacao = (alunos_caxias.TP_SITUACAO.isin(['Desvinculado do curso', 'Matrícula trancada', 'Transferido para outro curso da mesma IES']))
nivel_academico = (alunos_caxias.TP_NIVEL_ACADEMICO == 'Graduação')

nao_cursando = alunos_caxias.loc[anos & situacao & nivel_academico,:].copy()
nao_cursando.CO_IES.value_counts() 
# estudantes que iniciaram nas respectivas IES entre 2016 e 2018, mas sairam do curso

Universidade De Caxias Do Sul                              5181
Centro Universitário Da Serra Gaúcha                       3359
Centro Universitário Uniftec                               1096
Faculdade Anhanguera De Caxias Do Sul                       602
Faculdade De Tecnologia Da Serra Gaúcha - Caxias Do Sul     210
Faculdade Murialdo                                          194
Faculdade Ideau De Caxias Do Sul                            120
Faculdade Nossa Senhora De Fátima                            90
Name: CO_IES, dtype: int64

In [6]:
cursando.head()

,ID_ALUNO,NU_ANO_CENSO,CO_IES,TP_GRAU_ACADEMICO,TP_MODALIDADE_ENSINO,NO_CURSO,NU_IDADE,ANO_INGRESSO,TP_SITUACAO,TP_NIVEL_ACADEMICO,TP_SEXO
1,FBC7BF40B3F9AC1C46F78CFF70AC141B,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,28,2018,Cursando,Graduação,Masculino
31,CBDDDDED425BFD6FE4D0FE4FF02C51A3,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,30,2018,Cursando,Graduação,Feminino
66,38DE9F2553152BC720F5883A1CCA2F1F,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,27,2018,Cursando,Graduação,Masculino
112,9329D565D05B90609DA9CD08965CEC09,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,25,2018,Cursando,Graduação,Feminino
116,437C230EC285FF3C87C3403B0D609B35,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,24,2018,Cursando,Graduação,Feminino


In [7]:
nao_cursando.head()

,ID_ALUNO,NU_ANO_CENSO,CO_IES,TP_GRAU_ACADEMICO,TP_MODALIDADE_ENSINO,NO_CURSO,NU_IDADE,ANO_INGRESSO,TP_SITUACAO,TP_NIVEL_ACADEMICO,TP_SEXO
2,43ABA22A4C7DC2F6E69C71CE90D339A0,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,27,2017,Matrícula trancada,Graduação,Masculino
24,06358262B67CC5FC6B9DD6A79B3279C6,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,49,2016,Matrícula trancada,Graduação,Feminino
30,828155F75E4535A19FCCD13968A8351F,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,34,2016,Matrícula trancada,Graduação,Masculino
42,5739B97B827D9861EA3DC2CCA708F892,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,35,2017,Matrícula trancada,Graduação,Masculino
51,E5978EE98829531087113382015BC3EF,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,27,2018,Desvinculado do curso,Graduação,Feminino


In [0]:
def renomear_colunas(data, mapa):
  '''
  renomear as colunas para montar o antes e o 
  depois dos alunos
  
  argumentos:
    data -> pandas dataframe
    mapa -> dicionário
  '''
  colunas = ['ID_ALUNO',
             'CO_IES',
             'NO_CURSO',
             'TP_SITUACAO',
             'ANO_INGRESSO',
             'TP_GRAU_ACADEMICO',
             'TP_MODALIDADE_ENSINO']
  
  data = data[colunas].copy()
  data.rename(columns = mapa, inplace = True)
  return data

In [9]:
# É preciso renomear os nomes das colunas para poder juntar os dois dataframe da situação anterior e atual do aluno. 

colunas = ['CO_IES',
           'NO_CURSO',
           'TP_SITUACAO',
           'ANO_INGRESSO',
           'TP_GRAU_ACADEMICO',
           'TP_MODALIDADE_ENSINO']

novas_nao_cursando = ['ies_anterior',
                      'curso_anterior',
                      'situacao_anterior',
                      'ano_anterior',
                      'grau_anterior',
                      'modalidade_anterior']
novas_cursando = ['ies_atual',
                  'curso_atual',
                  'situacao_atual',
                  'ano_atual',
                  'grau_atual',
                  'modalidade_atual']

mapa_nao_cursando = dict(zip(colunas, novas_nao_cursando))
mapa_cursando = dict(zip(colunas, novas_cursando))

mapa_cursando, mapa_nao_cursando

({'ANO_INGRESSO': 'ano_atual',
  'CO_IES': 'ies_atual',
  'NO_CURSO': 'curso_atual',
  'TP_GRAU_ACADEMICO': 'grau_atual',
  'TP_MODALIDADE_ENSINO': 'modalidade_atual',
  'TP_SITUACAO': 'situacao_atual'},
 {'ANO_INGRESSO': 'ano_anterior',
  'CO_IES': 'ies_anterior',
  'NO_CURSO': 'curso_anterior',
  'TP_GRAU_ACADEMICO': 'grau_anterior',
  'TP_MODALIDADE_ENSINO': 'modalidade_anterior',
  'TP_SITUACAO': 'situacao_anterior'})

In [10]:
# dataframe da situação anterior
antes = renomear_colunas(nao_cursando, mapa_nao_cursando)
antes.head()

,ID_ALUNO,ies_anterior,curso_anterior,situacao_anterior,ano_anterior,grau_anterior,modalidade_anterior
2,43ABA22A4C7DC2F6E69C71CE90D339A0,Universidade De Caxias Do Sul,História,Matrícula trancada,2017,Licenciatura,Presencial
24,06358262B67CC5FC6B9DD6A79B3279C6,Universidade De Caxias Do Sul,História,Matrícula trancada,2016,Licenciatura,Presencial
30,828155F75E4535A19FCCD13968A8351F,Universidade De Caxias Do Sul,História,Matrícula trancada,2016,Licenciatura,Presencial
42,5739B97B827D9861EA3DC2CCA708F892,Universidade De Caxias Do Sul,História,Matrícula trancada,2017,Licenciatura,Presencial
51,E5978EE98829531087113382015BC3EF,Universidade De Caxias Do Sul,História,Desvinculado do curso,2018,Licenciatura,Presencial


In [11]:
# dataframe da situação depois
depois = renomear_colunas(cursando, mapa_cursando)
depois.head()

,ID_ALUNO,ies_atual,curso_atual,situacao_atual,ano_atual,grau_atual,modalidade_atual
1,FBC7BF40B3F9AC1C46F78CFF70AC141B,Universidade De Caxias Do Sul,História,Cursando,2018,Licenciatura,Presencial
31,CBDDDDED425BFD6FE4D0FE4FF02C51A3,Universidade De Caxias Do Sul,História,Cursando,2018,Licenciatura,Presencial
66,38DE9F2553152BC720F5883A1CCA2F1F,Universidade De Caxias Do Sul,História,Cursando,2018,Licenciatura,Presencial
112,9329D565D05B90609DA9CD08965CEC09,Universidade De Caxias Do Sul,História,Cursando,2018,Licenciatura,Presencial
116,437C230EC285FF3C87C3403B0D609B35,Universidade De Caxias Do Sul,História,Cursando,2018,Licenciatura,Presencial


In [12]:
# transição dos estudantes entre cursos da mesma IES ou IES distintas
students_flow = antes.merge(depois, on = 'ID_ALUNO')
students_flow.head(10)

,ID_ALUNO,ies_anterior,curso_anterior,situacao_anterior,ano_anterior,grau_anterior,modalidade_anterior,ies_atual,curso_atual,situacao_atual,ano_atual,grau_atual,modalidade_atual
0,6EE8F10AE6C23604BA9E66B50CBE8BFA,Universidade De Caxias Do Sul,História,Matrícula trancada,2016,Licenciatura,Presencial,Centro Universitário Da Serra Gaúcha,1427646,Cursando,2018,Tecnólogo,a distância
1,E18340E6B025CF3D7926D03F8B8A3655,Universidade De Caxias Do Sul,História,Matrícula trancada,2016,Licenciatura,Presencial,Centro Universitário Uniftec,Marketing,Cursando,2018,Tecnólogo,Presencial
2,2179F94C8A6F7ECE96775F3918D43A28,Universidade De Caxias Do Sul,História,Matrícula trancada,2016,Licenciatura,Presencial,Centro Universitário Da Serra Gaúcha,Relações Internacionais,Cursando,2018,Bacharelado,Presencial
3,2179F94C8A6F7ECE96775F3918D43A28,Centro Universitário Da Serra Gaúcha,Arquitetura E Urbanismo,Transferido para outro curso da mesma IES,2017,Bacharelado,Presencial,Centro Universitário Da Serra Gaúcha,Relações Internacionais,Cursando,2018,Bacharelado,Presencial
4,2E0E5FBF1352C5418152C9A0A55934F3,Universidade De Caxias Do Sul,Serviço Social,Desvinculado do curso,2017,Bacharelado,Presencial,Centro Universitário Da Serra Gaúcha,Direito,Cursando,2018,Bacharelado,Presencial
5,9AFC2902070B9B4E917AABBA7D3D161E,Universidade De Caxias Do Sul,Serviço Social,Transferido para outro curso da mesma IES,2017,Bacharelado,Presencial,Universidade De Caxias Do Sul,Direito,Cursando,2018,Bacharelado,Presencial
6,83BF48541638A5D1B3B38029E17B7266,Universidade De Caxias Do Sul,Direito,Desvinculado do curso,2016,Bacharelado,Presencial,Centro Universitário Da Serra Gaúcha,Direito,Cursando,2018,Bacharelado,Presencial
7,C411DFC070DDF83CB05E9F900E54891D,Universidade De Caxias Do Sul,Direito,Desvinculado do curso,2018,Bacharelado,Presencial,Centro Universitário Da Serra Gaúcha,Relações Internacionais,Cursando,2018,Bacharelado,Presencial
8,0F6434A66497D8E501D9214244410CE9,Universidade De Caxias Do Sul,Direito,Desvinculado do curso,2016,Bacharelado,Presencial,Centro Universitário Da Serra Gaúcha,Direito,Cursando,2018,Bacharelado,Presencial
9,DF4307F06DAF517C538658B9A0018A67,Universidade De Caxias Do Sul,Direito,Matrícula trancada,2017,Bacharelado,Presencial,Centro Universitário Da Serra Gaúcha,Relações Internacionais,Cursando,2018,Bacharelado,Presencial


In [0]:
# salvar em csv
students_flow.to_csv('fluxo_estudantes_2018.csv', index = False)

In [14]:
# gerar os agrupamentos entre a IES anterior e a atual dos alunos.
fluxo = students_flow.groupby(["ies_anterior", "ies_atual"]).size()
fluxo

ies_anterior                                             ies_atual                                              
Centro Universitário Da Serra Gaúcha                     Centro Universitário Da Serra Gaúcha                       364
                                                         Centro Universitário Uniftec                                75
                                                         Faculdade Anhanguera De Caxias Do Sul                       28
                                                         Faculdade De Tecnologia Da Serra Gaúcha - Caxias Do Sul     26
                                                         Faculdade Ideau De Caxias Do Sul                             4
                                                         Faculdade Murialdo                                          13
                                                         Faculdade Nossa Senhora De Fátima                           10
                                               

In [15]:
siglas = np.load('dicionario_ies_siglas.npy', allow_pickle=True).item()
siglas

{'Centro Universitário Da Serra Gaúcha': 'FSG',
 'Centro Universitário Uniftec': 'FTEC',
 'Faculdade América Latina': 'América Latina',
 'Faculdade Anglo-Americano De Caxias Do Sul': 'Anglo-Americano',
 'Faculdade Angloamericano De Caxias Do Sul': 'Anglo-Americano',
 'Faculdade Anhanguera De Caxias Do Sul': 'Anhanguera',
 'Faculdade Da Serra Gaucha': 'FSG',
 'Faculdade Da Serra Gaúcha': 'FSG',
 'Faculdade De Tecnologia Da Serra Gaúcha - Caxias Do Sul': 'FTSG',
 'Faculdade De Tecnologia Tecbrasil': 'FTEC',
 'Faculdade Dos Imigrantes  Fai': 'FAI',
 'Faculdade Dos Imigrantes - Fai': 'FAI',
 'Faculdade Ideau De Caxias Do Sul': 'IDEAU',
 'Faculdade La Salle - Caxias': 'La Salle',
 'Faculdade Montserrat': 'América Latina',
 'Faculdade Murialdo': 'Murialdo',
 'Faculdade Nossa Senhora De Fatima': 'Fátima',
 'Faculdade Nossa Senhora De Fátima': 'Fátima',
 'Faculdade Serrana': 'IDEAU',
 'Universidade De Caxias Do Sul': 'UCS'}

In [16]:
# montagem dos dados para colocar no flourish
fluxo_ies = pd.DataFrame(list(fluxo.keys()), columns = ['De Onde', 'Para Onde'])
fluxo_ies['Quantidade'] = fluxo.values

# remover grupos iguais. Exemplo: ucs e ucs; fsg e fsg (esses são alunos que trocaram de curso, mas não de IES)

fluxo_ies = fluxo_ies.groupby(by = ['De Onde','Para Onde']).filter(lambda x: x['De Onde'] != x['Para Onde'])

# trocar os nomes das IES por siglas usando o dicionario 'siglas'

fluxo_ies['De Onde'].replace(siglas, inplace = True)
fluxo_ies['Para Onde'].replace(siglas, inplace = True)

fluxo_ies

,De Onde,Para Onde,Quantidade
1,FSG,FTEC,75
2,FSG,Anhanguera,28
3,FSG,FTSG,26
4,FSG,IDEAU,4
5,FSG,Murialdo,13
6,FSG,Fátima,10
7,FSG,UCS,118
8,FTEC,FSG,55
10,FTEC,Anhanguera,7
11,FTEC,FTSG,2


In [0]:
# csv
fluxo_ies.to_csv('quinta_e_sexta_visualizacoes_conexoes.csv', index = False)

In [18]:
# criação da tabela dos nós

cursando_2018 = cursando.CO_IES.value_counts()
cursando_2018

Centro Universitário Da Serra Gaúcha                       4370
Universidade De Caxias Do Sul                              3001
Centro Universitário Uniftec                               1859
Faculdade Anhanguera De Caxias Do Sul                       410
Faculdade De Tecnologia Da Serra Gaúcha - Caxias Do Sul     111
Faculdade Murialdo                                          100
Faculdade Nossa Senhora De Fátima                           100
Faculdade Ideau De Caxias Do Sul                             78
Name: CO_IES, dtype: int64

In [19]:
iniciaram_2018 = pd.DataFrame(zip(cursando_2018.keys(), cursando_2018.values),
                             columns = ['Nome da IES','Iniciaram em 2018'])
iniciaram_2018['Nome da IES'].replace(siglas, inplace = True)

logos = ['https://upload.wikimedia.org/wikipedia/commons/4/4d/LogotipoFSG.png',
         'https://www.ucs.br/site/static/img/capa2016/logo_ucs.png',
         'https://www.ftec.com.br/static/front_end/_images/geral/logo-grupo.png',
         'https://i1.wp.com/tempofm.com.br/wp-content/uploads/2018/10/anhanguera-2.jpg?resize=945%2C630',
         'https://qb-assets.querobolsa.com.br/logos/colorido/large/159/logo_1489433583.png',
         'https://www.sinprocaxias.com.br/igc/uploadAr/FileProcessingScripts/PHP/UploadedFiles/thumbs/faculdade_fatima-sinprocaxias.png',
         'data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxIQDQ4NEA4QEA0NDw4QDg8PEBAQDw4PFhEXFhURFhUYHSggGCYlGxUTIzEhMSksLjouGB8zODMtNystLisBCgoKDg0OGRAQGy0gHiUtLS0tLS0tLSstNy0tKy0tLi8vLS0tLS0wLS4tKystKy0tLS0rLS0tLy0rLS4tLS0tLf/AABEIAOEA4QMBEQACEQEDEQH/xAAcAAEBAAIDAQEAAAAAAAAAAAAAAQIEAwUGBwj/xABLEAABBAECBAIFBgkJBgcAAAABAAIDEQQSIQUGE2ExURQiQYGRBxcyZHHUFTM1QlJigqGxIyRUcnN1krKzFiVTtMHRNDZDRJWipf/EABsBAQEAAwEBAQAAAAAAAAAAAAABAgMEBQYH/8QAPxEAAgIAAwMGDAQFBAMAAAAAAAECEQMSIQQxURMiQWFx0QUUFTJSgZGhorHS8FNiweEzNGOCwiRCcvIjk7P/2gAMAwEAAhEDEQA/APnd9l45+iWL7KixfZBYv7ULYtQWLQWLShaLaULQtC2ghbCAIAgCChSEpCkFCu6CmEGovsgti0GZC0LaCAIBSEpCkFMINRaCxaC0EKEIEAQBAEAQBAEKEApCUhSDKhSChXdBQQahBqL7ILYtBYtBaCFCAUgpCkJlQpBTCDUWgsWgsIBSChSEpEpWyZUKSxlQpLGVCksuVClLGUUqKFKChXdBT4lQtMfBBqEGovsgti0Fi0GZC0LaCAIAgFITKhSCgg1HuQWxaDMELYQCkJSJSEoINR7kGoVJqEGo9yC2L7KFt8BfZUW+AvsoLFoLFoWy2gzIWgtBC2EAQBAKQlIUljKhSCgg1CC2LQWLQWghQgFISkSkJQ3Qai0FltC2ggIgCAqAIAgCAIAhQgCChSEpCkGVCkFD3oKYQaj3ILZLQWW0GZBChAEISkslIV3QUEGotBZbQtoIAgJSWTKhSWMqFJYyoaUsZUNKtjKhSWMqFKWKFIKLXdC11hBTCDUINRfZBb4EtBmFpQzItoLCFCECAUhKRKQUEGovsgti0FlQoQCkFIlK2Shug1F9lBb4D3Ki2PcoLfAX2QW+AvsqLfAX2UF9QvsgsWgstoXMhaDMhaC0EKEAQgQCkFIlJZMqFd0FDdBqL7ILYtKFltC2ggCAUhKRKVFDdBqLShYtQWioUIAgCAIAgCAIUIAhBSCkSksmVCksUK7oKG6DUINRfZBYtWhaLahbQQBAKQUiaVbJlFIKYtBbGpKGZBQBCikJQpWyZUNKWXKhSljKKVsZRX2oKFd1BRu4PCMifeGCWQfpNadP+LwUlOK3mueLCHnSo3zyhnAX6M7/ABRk/DUsOWhxNXjeF6R1mZgTQmpYnxn2a2OaD9hPis1KL3G6GIp+a0zgY0khoFuJAAFkknwACyM22tWWWNzHFjmlr2khzXAhzT5EHwSiKVq1qYX2Qt9QtKFi1KFotoWwgCAIBSCkSlbJlFd0FMboNRaUMwtShmQQoQClSUhSEoiDUt9kF9QtKLmGpKGYakoZkcmPE6R7Y2NLnvIa1oFlxPsUempjLEjFWz6jyxyJHCGy5IE0+x0HeKPtX5x7nb+K5J4rlotx4u0+EJS0hoveezZj7AAbDwA8AtWU814hTjplIsQ1snEY5jxJo6QY9z+qB09DWlztV7eAKsMNylS0ZthiSTWW70qt+pp4vLmI3Gb/ADaKSMzvkjdd6bawgxStNgW0nYkfwU2vbMbZcOEmk9Wmn2JqmvX1GyWPiyxHzmnST9+9Pus8JnZ0mRxPLw54osxnpWcIvSHGKaGNj5X6WZLQXNAa00CHN2ApfdYuybPi4McSSrRarsR4uz7bj7PKsOXq6Dr87lUFwGO9zJHn1MXM6cUrz5Qzg9HI9v0XA/qrxsbwdOKzYbzLqPo9l8O4c+bjLK+PR3/M87l4skLzFLG+KRv0mSNLHD7QV57TWjPchiRms0Xa6jhUMhSEpE0q2MqFd0FDdBqLQWxqShmFqC0EKEAQgpUmUiCmLQWy2gzIIUICqFCAID6Z8mPAA2M5z2/ykmpsF/mxjZzvtJsfYO65saVvKeJ4S2m3ya3LefRY4lhGJ4spHOI1uWGa7BjR4Ys4zAHHQfB9sP2OGk/uKmGqxI9pmpuOq6NfYeF5DjLGZcZ2p2OS32B9Shxr3N+C8/wwn4uuqS+Uj6PwpJSlhyX5vZzaOkwIb4xxSWvxT86j+s/J6X+V8nwX6DivLssI9UfkfE/7mdqR6rmbFj9nscA+N48nMdbXe8Feem4u1oZGObBFPinElD2M1NfFIwiUwEAjS1kt6WmxbWuYPVFVuDjjrllzt/Gjr2La5bLPNHXqt/fzPAcx8HdhZLsYyslLWsdqYHNID26mhzXC2miCRv4ry8SGSVH2WybT4xhLESo6u+ywOm3wGpKGYalKGZBAEKEApUlCkJlQrugp8SboNRaDMW0LYQBASkJlQruUJXWXdC6jdBqAD4JoNT9D8JwxDDFC36MUbGD9loC4Vq7PkMablJyfSdpG1dMInM2bDI11Rga2yujVlhhM4HijfkVzyjTszTPJ8Ex9OXns9hyWtH2dWUBcXhWGaKhxml8z29oxLwcKX5b90TzXLrNbMzK/peY8t/qt1PJ95nH+FfabY6yw4L7+R8tHidiY1xGQD2Qsly5hcOK0SOb/AMR91HF+06vcHLGc1GNm/ZsCWPirDj0ng+I8DzZXuy5hEH5I9JJflYrHObINYcGGSwCDsK7LzZQm9WfZYW04EEsOF6abpdGnA6vD4XNNDPPHEXxYoa6dwI/kw4kA0TZ8D4eSwUW02jpnjQhKMJOnLccfD8CXIlbBBG6SV/0WNFnuewHn4KRi5OkXExYYcc03SN88tZHViha2GSWd5jjbFk40tyAWWkseQ2u9LLk5XRp8bwsrk7SWuqa09aNDB4dJPL0YmapNMjtNtb6rGlzjZIGwaSok26RtxMSGHHNLRd4n4dJHDBkOYWw5PU6LtTTr6btL9gbFHzRppJkjOEpShF6qr9Zt4nL+RJCycdFkMpe2J02RjwdRzSA4NEjwTRIWSg2rNc9phCThbbW+k38kanEcCXHlMM8To5GgHS72g+DgfAg+Y2WMotOmbMLGjiRzQdoyyuGTRQwZEkTmQ5QeYHmqkDDTtrseI8a8wq4NJMkNow5ylBPVbzZxOATyQsyAIWQyl7YnzZOPB1XNNODRI8E0SFVhyaswnteFGTjq2t9Jv5IxHAcn0tuD0HDKeQGRuLW6rFghxOkggHe6UySvLWpfGsLk3i5ubxJxDgs0DBJJ0dJcGjp5ONMbIJ+jG8n2HeqRwa3lw9ohiPLG/Y180daQsTdSFISgg1FoLGpKGZFUMyoDOF+l7XfouB+BtCSVpo/RuObAI8DuPsXHA+LmbsS7cM0yNyJdsKNMjKWlZ0RGpIPZ5rjmr0NqPHZuT0YuK5Q+kZphFW5MupzY6Ht9aZiyjg8vtmFDoUnJ9kf3kd+2YmXZ4L8qXtruNXDwOhBDjbXBGGurwMhJdJ/93OA7AL2caeebkeKlRmISSGgEkkAAeJJ8AtRTwnP3GmyPbgwuDoMZxMrx4TZX0XOB9oaPVH7R9q8/aMXM6W5H13gjYeRw+UmudL3I5ucxi/zTqek+k/grB0dPpdG+gNOq9/HxUxMul76RlsSxbnlrLnlxvedpwmEYcfDsd8+MyOQST8ThmnayR7MmPQGFhF+rCQQPN5Wceakvb6znxm8aWJOKk2tItK/Nd7+tnXYfDH4ePzHjDfKgZjxtcB67sMz/AMq8V7HM6ZPYrFRyqa6f0N88ZY09nm/NdvqzVp607o6r5O/yzgf2w/ylYYPno6PCV+Kz7Dl5F/KbtvCDiF9v5rKrhef7THb3/p12x+aJx8/7m4H9nEv+ZCk/Mj6xs0l41jf2/Icb/IvBv7Tin+pEk/Mj6zLZ3/qsb+z5M7DJ4c/LwuXsb/3E5zIw527m4wnGhzvbpaOoR2BWTi5Rgu00QxY4OLtE+hZX66+b0Oz4hA3Mi4jixz48kbBHPwuGKcSSMbjR9MsDBvb4QSR+k1ZtZk1fZ6jnwpvBlhzcWnqptqlzne/qfuPOcc/IvBv7Tin+rGtUvMj6zuwF/qsb+35M9Ny1vJyqX/jdXEBZvX0A89IG/Z+MrstsP9nrPP2lKtqrdzfb0/pZ4LiPouhvo3pOu/W6/S06a9mje7paJZeg9fC5e/8AyVXVZ199lgb7YtKGZBClQBANKWXKhSWTKhpSxlR9x5D4sMnAhddyRAQy+epgoE/a2j71ySWWR8xt+DyeK10PVHqo3LfCR5zRstkXTGZraDpFZYgSNeafQ10pFiMaq/Sd4NaO5dQWGHzpXw1NkYZmo8ft+48ucckxNJtmI8yOPsmzt9+4i1O39r9I8WOC9DZsPk05vzpKuyP7t+yjDa8blcTTcjJ0a2nKYcRwZeg5kZnimmZTZYsSafpxOsOILdg4jYb7Ak+JBGnFk6yq/YehsGDFyWJPK0uhyUdfX0Hifm5P/Gy//i8n/uuPke32M+j8oP8AJ/7Im7ncmSzviklnyXGGKGFg/BOQB04xTWkXvssnht779hqw9qjhpqOXVt/xI72YcT5JlyZ5MmbIy3Syu1PP4LyAPCgAAdgAAPcksNydu/Yy4W1rCgoRUKX9SJu/7PZfpEWUMzLbPDCyAPHCZ/XiaNIEgJqTbY3d0Fllld2/YaeUwsjg4wpu/wCIt/Vw9Rhm8rSyPika52O+F/UY7F4JLA7WCCHFzTe1Ch4dlHC+r+0sMdRTTaknpripmzlcFyZGSt6j435DXMyJoeBvinyGu3eHvb+kfGqv2qtN/wDUwhKEWno0tyeKmlwpdXQceNy7MzHhxi50sWP1OkJ+BPlLNbtTqLt9yii0q/xLPEjKbnom99YqW4uNy9KyJsDrmijfI+Jk3A5XiFzzb9G+wJA28Ngqk0q/xJOcJSzJpPprFWtHEeW8ozvyTmZZmfA7HDvwTMBFC5unTE0Go6bYFAVZ81Mru9fYXlMPIoJRq7/iLV9fH1mnwvkiXGnjyYcnLbNE7Ux34KyD7KIIJ3sEj3rGOHldq/Yzbi7VysHCeRp/1InZY3L8zImwOJmijfI+Jk3A5XiEvdbwzfYE1tuNgsknVf4mmWJFyzJpN1dYq1ricLeW8r0yPPOXlHIhoRE8Jm0RtDS0MawHS0AE7AUmV5s1v2GXK4fJPCqFPf8A+RfM4+IcoPmYIzcYDg7VBwJ8L9gRWptGt/D7FHC/+plh7RybtNPtxUzzeVym2BxkkmL8WIgZFRSQZGOXD1DJDILDXGgHCxZqwteRcTs8Zm0ko03u1Ti+NNaWuG88vKBqcG7ts6T5i9vYP4LUdq3amGlLJSJXdUUxv2QamWpSjLMhaUMyFqUXMjveUOY3YGRr3dBJTZ2DxLfY4dxZ/ePasJwzI5Nr2eOPCulbj7dw3iEc8TJonh8bxbXN/h2PZc6bi6Z8xiYUoSyyWpvCRbFiGnKcrWk0TsD4WDZ+weJXTh4U59SMbRq5pBoWfVNhrTuHVWpzh4H2CvDet6LfRwsKMFX36zVLFpNR6d7/AERoOj8BQAAAAAoADwAHsW5uzQeZ525i9AAhY3Vmyx9RgcLZBGXOaJHXs42xwDe1u9gd2bJsvK86XmmMnR5TN5ZlPDn8Xi4h6VFet40zMmIM5idIQ83s8Ps/qkr0I40eU5Jxr2cLMa0sw5P5Un4kyaRmUIWwEN9cyPMjtOpwAB/NBZf9cJj48MFpON2ErOuwOD5OU+QYLn5ccf57ZWQuc2gdfTleHgesBdUtksSEEuU5r9vvSJTe418XByZZzix2+cbaBkw047bNeX6XncbAk+PkVk5QjHM93Y+4mptYvL+fK+WOON0j4PxrWZeMTHsDZ/lfDcb+Hs8QVi8XCSTb39T7i0zXzuGZkETJ5WStgl2jmbI2WFx8hJG5zfYdr9h8llGeHJ0qv74imc2PwPOkxzlsY52MBbpvSsdrG+GzrkBadxsQDuNli8TCUsr38KfcKZp8Mx8jJmZjwdWWaTVoYHkE00uO7iANgfErObhBZpaIm84YHSve1jXu1uNAPl6YvyJeQG+8hVqKVsG9xfhOZiaPSWPh6n0AciJ7nDffSx5NbHeqWEJ4c/N19X7FpozxeC5ssByo2l2M2tUpy8djGEmgHapAWmyNjXijxMJSyvf2PuFM487heZBE2eWOZsDzTZmyCWEm6A6kbnNG/dIzw5OlV/fEUzZwuDSuxPT58n0XCL+lHK8ySPnl39SKJu7vous2BsfI1jLEipZIq3972K6TYj5eknxpsnCzTljFAORDolgyY2kEh4YXODxQd4H801Z2UeMoyUZxy3uejQrgdxyvO6bhruq90hY/ieK1ziXO9Gdw10/Ss7kCRgcPL2Lx/C2HGOLFpVad+/uPa8FTlyeJHoThJdudK/Y2j5zZXhH12o1JRM3EWhbRUKVYmZUAQEQh23L3MM+DL1IXAtP04X2YpP6zQR8RRSk3qrOfaNlw8eNS9q3n1HhHynYcjR1Q7FkoarZ1I7/VewaviAuvCngrop9n2z57aPBO0xfMqS7dfY9Duv8AavCcL9Px6PjcgaT9oO5966Vi4fE8+Ww7VucGdXn89cPiB/nBmcPzYI3uJ/adpb+9Yy2iCN2F4I2qe+Ndr+2edxflBlyeI4UEEYx4JMzGY8kiSaRhmALS6qYCD4AX3K0+MSlJJaKz0vI+Hg4M5zeaSi+pbvvuOg58/G4H92Qf8zkL6HwT/LLtfyR4nhX+bxO09PyVlMflN4PJtFm8ExoXtPiJ3ROyS0Dz0ZUp/ZCu0Rajyq3qTfquv0RxLgZ8iPdh8XweFvI6keHkjJA/pc+mdzT/AFYooB7iptNYmFLEXFV2LT5tiOjo0/k1xelxnisAFOhxeIQsHZmTG0fwCz2uV4MHxa+QjvPG8n/lDhmn+mYNfZ1mf9F2bR5k74P5GC3o9n8mgB4rxaiA042dR3oN9IbR27Li2v8AhQ7V8jOO9mpxFjsTlfGxg5uTFxHJ63pMRd0IAwtcIm6gHaiYiSC0f+p7RvnFrE2py3ZVu6fvX5E3ROPhH/lLi/8AeWL/AJsRZT/mof8AF/5Beazi5Ce/GviLOjqOTBijrTwQVjhzZctzeq9ocdIhaKsjWU2lKfMfBvc3ruW71iPE0vlQ4QMXiWUGgdHKacqIjdpbJZeL8Pph+3kQtmx4mfDjxWns/YklTO3+WH/x2J/d0H+pKtOweY+3uLPeY8AYRy1xtpBa5uTjBzXAhzSHxAgg+BHkmL/M4fY/1C81nP8AJOeoziuHJviS4ZklafoNeLbq7EtPj+oPJTbtHCS32WHScnPw/wBxcvFn4kwR35dQ47CPf+M/eps38fFvff6iW5HF8ij3DiszRfTdhSl/lYmh0k/F3xKvhBLkk+m/0ZIbxyrp9DyenXS/CHFOnXh0/wAEy6f3UvP8LXmw731+jPX8Febi/wBv/wBInzleAfYhCEIQjSJSWTKjO+yhst8BfZBb4C+yC3wJfZKF9QvsgzC0GYakoZkNSUMyO25ScBxPh5JAAzcQk+Q6zVnh+cu059ra8XxP+L+R3vOwDZeHTSxOfjjDGO8a+lqmiyJ+pFro6SNTT4eBX0/gl3s+WL1TfXwPjfCqa2ube5012NWjr8nmhh4jBxKGBsEsL4nuZ6QZWyCNrWBoJaNAMbdJ8fFdy2d8m8OTtdlfep5962XhnNYi4lJxSSET5DpZJo2+kGJsbn6gQaadYDXaQNvBJ7O5YfJp0uyxetnMOdOnxT8K40LIJXl/XidMZo5g8+uPotLboHb2tB7KeK3hcnJ2ujShm1s14uYMaGc5WLhCGcajDryutBivII1xx9NpNWdIc4gbbbBZPBnKOWcrXZTfrsWjPlHmpnDjM8Y4nkmjdC5zsoxgRO0kjTodZtt6r9vgpj7O8WldV1fuFKjm4HzZDDj5XD5oDLwzKOpsPXaZsV+1PjeW0TYafAbi/abmJs8pSU4upLpreFLoMcXmuCLh2RwsYhfBkytlkldlhkrpGlml+kRkN/FM9Wz7dz4o9nm8RYmbVdX79YvSjr+J8YgmhxYGY/RbidQAnKEvUEjy95IMYok6d99mgVstkMKUW23d9VbvWRs7LjfN0GZj4mPLg+rhR9KJ4zz1DFTQWOd0t7DBv4jxWvD2aeHJyjLf+X9yuVnPxrnXGzJ4cifhrXPgjjjaBnvawsY4uAcOlvuTaxw9lnhxcYz3/l/cOVnHFznEMXNxHYepvEZ35GS8ZulxkMvU9QdIhoBA81Xs0nKMs25UtP3GY0280NixJsPEhZjMygG5UpmM+ROwAgR6yGhraLtg32nzN58g3JSm7rdpSFlwua6wTw3JjZlYYf1Im9XozYz7J1RyAO83bFp+kR4GlJbPz+Ujo+y79QvSiwc0tx8efHwovRzlNDJ8iTI6+S+MX6jCGMbGNzuG3v4+FHs7nJSm7rcqpfrYvgd9yews4W5zgWtfNxSZhrZ0LOFOjdIPMB7g2/PZeP4XkuVgulJ37z2fBMXkxX0PIvXni/kmfO6XgWfW0N0GovslDNxGpKGZGaxNwQBAFQEAQgQBAAaNjYjcEeIKEaTPb8N590gmZk7Zn0ZpMWSJrMh4AAkkhlje3VQ3cKvyW6OLTvVPqPOxdhtKNRkluUk7S4Jpp11O6N35wY/r3x4d93WfjD9KXu7jR5MX4eH8f1j5wY/r3x4d93Txh+lL3dw8mr8PD+P6x84Mf1748O+7p4w/Sl7u4eTV+Hh/H9ZPnBj+vfHh33dXxh+lL3dxPJv9PD+P6h84Uf1748O+7p4w/Sfu7ieTv6eH8f1k+cOL69/+d93Tl36T93cPJ6/Dw/j+sfOHF553x4d93Tl5elL3dw8nx9DD+P6y/OFF9e+PDvu6eMP0pe7uL5NX4eH8f1j5wo/r3x4d93Txh+lL3dw8mr8PD+P6x84Uf1748O+7p4w/Sl7u4eTV+Hh/H9Y+cKP698eHfd08YfpS93cPJq/Dw/j+sfOFH9e+PDvu6eMP0pe7uHk1fh4fx/WPnCi+vfHh33dPGH6Uvd3Dyavw8P4/rHzhR/Xvjw77unLv0pe7uJ5OX4eH8f1HQ8wc4OnZJHEyRnWAZPPPKJsiWJp1NhGlrWxMvcta3ehZK1Sne72vedWFs+Ws1JLVRiqSfHe231t6dB5e1ro7MyKhQgIgMt1NDPUboOcN0HOG6E1G6DUWfJBb4C+yC3wF9koZuompKGZDUlEzItpRbQtC2VAEIRAEAVBKSzHKiUgyjdBqL7IS3wGpKGYWhbRUKEIQoSiUqTKN0Go3Qls5FidAQgQBAEAQBAEIEBEApCUiaQlkyoaVbJlQrugp8Rug1Jugti+yDN1DUlDMi2hbCAIAhCEIRpMlKky8Bug1F9kJm4oakouZFUKEBlXdQzp8RR80FPiKPmgp8Rv5oKY3Qak3TQmo3TQaiz5ILfAX2VFvgNXYpQzDUlEzIWlFzIWgtFQBARAEAQhKSyZUTSlkyjfzV0Gos+SC3wGpKGZC0FoqhQgIVSUTShjlQrugp8TlWJ0BAEAQgQBAEAQhEAVAQEQgpLJlRNKWTKhXdBXWN00GpN00JbGrslDMXUlDMhaFsIAgCAhAVsxaRK7pZMvAbpoOcL7K0M3FDUgzItqFszrusTbXWK7oK6yV3QU+IruhKfEV3QU+Io+aop8Rumg5w3TQnOJZ8kFsX2QW+A1dilEzdQ1JQzIakouZFtBYQBARAEAQhNKtkyoae6WTKTdNCc4X2QW+A1JQzItoWwgIgBCEaRNIVsmVHMsDpCECAIAgIhAqAgCECAiAICUhKQ0hLJlRNKtkyoV3QU+I3QnOFnyQW+BNSUMxdSUMyFoUIAhAgMSFbMXFCu6WSuDG6aC5E1JQzcRqCUM6OWu6ws311iu6WMvWK7q2MvWK7pYp8RXdCU+Irugp8SboKY3TQc4bpoTnCz5ILfAX2Shm6iakomYuoJQzIWlFtBAEAQBARCClSUiaVbJlRK7oSnxG6DnDV2SiZuKGoJQzItqGRFSBAEBzLA6AgCEIgCoCAIQiAIAgCECAiEJSpKQ0pZMqJp7pZMvWKPmmgp8RumguQ1dlaGbqGpKJmQtQtoIUIQICFUjJpVsxyoUfNCU+hks+SC3wGrsUoZuo5q7rA6afEV3SxT4iu6Ep8RXdBT4krugp8RR80JT4jfzTQajfsmg5xN00JchZ8ldBb4C+yUMz4DUlEzDUEoZkLQuZFUFkQBUBCBARCEpWyZUTSlky8BR80JT4iz5ILfAalaGdFtQWiIUIAhDnWB1BCEQBUgQEQBAEAQhEAQgQBASlbMaRNKWTKhp7q2TKSj5oKfEbpoOcL7JRMz4E1JQzotpRcyCAIAgCEMS1WzFxRNPdLJl4MbpoOcLPl+9CW+BtlYm4xQpQhCoQxKFIhDIKkBQGBUKUIQqoIUIYoDIICqkMSoDFAZBUhSgMHKBnGVmjRIyYjETkUNoQBCEQBAEIEB//2Q==',
         'data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxITEhUTEhAWFhUVFRUXFxUVFRUVFRUVFhUXFhUWFhUYHSggGBomHhUWITEhJSkrLi4uFx8zODMsNygtLisBCgoKDg0OGxAQGzAmICUvLzUzLS0tLS0vLS0tLS0tLS0tNS0tLS8tLS0tLS0tLy8tLS8tLS0tLS0tLS0tLS0tLf/AABEIAOAA4AMBEQACEQEDEQH/xAAcAAAABwEBAAAAAAAAAAAAAAABAgMEBQYHAAj/xABFEAACAQMCBAQCBwUGBQIHAAABAgMABBESIQUGMUETIlFhMnEHFCNCUoGRYnKCobEVMzRDY5IkU6LB8LLCFiVEc4O00f/EABsBAQADAQEBAQAAAAAAAAAAAAADBAUCAQYH/8QAPBEAAgIBAgQDBAkCBgEFAAAAAAECAxEEIQUSMUETUWEicYGhBhQyQpGxwdHwUuEjJDM0YnIVQ6Ky0vH/2gAMAwEAAhEDEQA/ANKh3oBzEcbUA4C9xQCyb0AohxsaAOV7igDjegOVsbGgBZcbigBG9AcDjrQHMMUAPWgABoDiKAHrQAA0BxFAd1oDgaA4igAzQBTQBTQBCaATYUAkxoBvIKAaIO4oB3Gc0AtGcbUAvjuKAVU5oAyHHWgDEdxQBxvQAKcbGgBK43FAD1oAFONjQHEYoAetAADQHEUAPX50AANAcRQAnegABoArCgOzQBDQBWoBImgEnFAIOaAZwnFAOlHcUA5jOaAUQ460Arp7igFF3oAVONjQBivcUAYb0ACnFAcy9xQA9aA4GgAIoAetAADQHEUAPWgABoDiKA7rQBQaABhQAZoBNqAI1AItQCEgoBpEc0A5j2oBwB3FALpvQB0OOtAKFe4oAw3oAVOOtACR3FAD1oDgcUBzD0oAetAV/iXFnMhgRJEIYapPIPs9LMCmGJAYjSGx2cbECsfi/Enpany/a7Z6fxeRNVXzPcbLxF7ZhtLKjA+XV4jq4KhQrSNk6tRGnJ3C4AGTWbwXjNl2a7t328/4iS6lLdFnQ5AOMHA2OMj2OCR+lfVFU4GgBIoDutABmgOIoAOtAFBoAGFAEJoBJqASegEHoBogoB1Gc0AvHtQCwHcUAqu9AGU4oA5HcUAYb0A2v+IRwIXlbCjpgFmJ66VUZLHY7Adq5lNRWZPCA0teMhmUGNk1acZZGKlxqRZVViY2YZxnbO2ckA9AlutAAD2oCDuOPZZUhU5bWdUiPowmB5SNmJzsMg4UntWRreMU6eMnH2mnhpbYJYUuXUZf2gZ5FbSvkSRS6MHRtTR4CnsfIcg9NtznNYPG+I16nSwXK1LPR9Usfkyemtxkw08hRo5AhfQ5OkEA7xugOT2BcZPYZPas7gOqq0+oc7PJ/j/ckvi5RwhSDmBsoSEkWQsB4AZypCk7tnSVyNOo6QCw9cV9Pp+OwnOSujyJd3+2CtKhpbPJPWlwJUDrkZ7HqMHBB+RBFbsJqcVJdGQNYFQa6Ay4jxBYcZGSQzdVUKi41O7sQFUal999gaASseMxyyPCcpNH8UbdxpVtUbdJFw67jpqAIB2rzmWcAezTqiszsFVQWZmOAqgZJJPQACvQRsXH4SVyJEDsFR5InRGZjhV1MPKScABsZJAGSaAlOtAEzQBWFAJmgEXoBCSgGcW1AO1FAOIzmgFUOKAVx3FAKKc0AKnFANuK8Qit42mlcIijcnv6ADuT0ArxvBzOagssx8c0SX3EYy50RMJYo0ztGJInVSfVydIJ98Vk8UblppNdsP8ABrJT0mp8TUrPTcuVk5knUaSGkMUo22ZVuInlyw6GEwmMg9C6D7wq1poN2zuTzGajj4I0ZPZLyL0R3FXTg5txt1oCmX9vMqtE8XnnkYIEOYmL+ZkByXVcB2YsNstjOAK+X1nCb7NcroJKPmn382vP3FmFsVDDGcHFI0kRppIYjKk3+YArrHMFgKlsasozHON8/KsXVaKbjKuvM+SS367te18yaNkVhyeMit9xe1kCxC5hYySRroWVGLAuuV0g7g9CPQmq2l0eoqk7JQaxF746PB7K2D9lNAWLyszZjw06FV8EjHiQtIkrEyYCyAFPKc5CbagpxsPgttsKnW1KPXd9nh49e5H4yWclv4TBJHEiyeGGUYxEGCKBsANZJO3c9a+xgsRSKbHvWugVnmolCG8Nn1mBVUDOXScNo9iQ2oE7fY7kVV1lMrqJVxeG0dQeHkoHO968C24jkxJFIAsinqbaBYZG37a2dcHqFPaqcJ82rlj7sUvjnJV183ClY6tlv4PzEOJ2bKoAuU8NnhyAJPDkR8KT9x9Ok56asH1OpGSY0+oVsfUirHhblpVRJC9yLlTrgeH6qZboy6zJ4SrLhWDZZmYNH5CyuSOiwaO1AFO9AJ5oAjigEmoBvJQDSGgHUe1AOFFALJvQCinFAH09xQFB5w+khYGMNqiySLsztkxo34QBu5HfcAe++I5WY6FC/WqD5Y7mZcc4/c3bBriUtj4V2VF/dUbfn196icm+pm23Ts+0yNRyCCpwQQQR1DA5Uj3BANctJrDOITcJKS6o1Xlvj63Sgo2mceaSIMEbWBpM1uzAjcdVYFTsGx8RxK9RbwuXJYnKrs11j6M+mpthqY80dpd0WyHj7AYZ4M+k7PaN8sFXV/3lOPSt2jWUXrNc0xKEl1QEnMBPwyW+f9Fpbxh8440XHzJxXVuqpqWZyS+IUW+iILjnGEhUyXDsmpSuGZWuplzvFGqeWGM4GdJyRgsykZrGt4pPVN1aJe+b2SOpKNS5rGZTxvij3MxlcAbBUQfDHGudKD9SfmT06Vd0mmjpqlXH4vzfdnz+r1LvnzduwxUkEEEggggjYgjcEHsRVhrKwytGTi8o07lnmdLgaGKLO2NcTkok7KMLLDIu8UwAG674HQ4BXGU7uGPCi5U+nWP9j6SjUV6ldcS/MuEHG3TZ3Ax2uVMWPc3MQaI+wwD61rafiWlvWYTXx2fzJJVyXVC3/wAQf6lmPldmQ/kgjBb5Zq3KyEVmTSRzhkPxfi+lfGllKKuQJmQxYyMFbW3OX8QjI1vlhvpGGNY+o4spPwtJ7c/PsvVsk5FFc09kZJx7ipuZdenQiqEij/BGvwj5ncn547VPpNN4FfK3lvdvzZga3U+PPK6LoNLO6kidZInZHU5DKcEfn/2q1nBVjJxeUX7gH0pTKyrdoJE2BkQaZB+0VHlb5ACpFY+5fq18k8TNVsrtJUWSNw6OMqw6Ef8Anapk8mnGSksoVIodBDvQCZNAIyCgEXoBnGKAdRnNALptQC4HpQCq70BXef8AiMsFofBUs8kkUQwSDiRsEAgggkZUEHYsD2qO2XLBvOPXy9SOybilhZM34bw66meWNOD2Z8EgSKFCMpYZVRJ43xY/TvismrSOyCnXbP35X5NELnPmcXVEjrng0cjtFFFJb3Sg5tJWLCTAyfAlbctjfS3XscDNeu27Tv8Ax94/1JYa/wCy8vVEEtNVflVrll5Mr3/nv+Yq8ZcouLwwy5BBBIIOQQSCD6gjcH3FRznH7LPpuGfRzU6nTy1EXyv7q8/52J6y5zvoxjx9Y9JVV/8Aq2Y/mTVCzhGksfNy4fo8Ga+I6qmTrn1Xmg13zxfOMeMEH+mij+bZI/LFeV8G0cHlxz722cz4te+mEV+aVnYs7MzHqzEsx+bHc1pxjGK5YrC9DPstnY8yeQtdHB1AARQ9Ta3RNcO5qvYQAlyxUfdkxIP1YFgPYEVn38M0tzzKCz5rb8i9VxK+vbOfePZufL5hgSIme6Rrn/r1D+VQR4JpE90372yZ8Xua2SRAXt5JM2uWRpG9WJOB6KOij2GBWlVTXVHlrSS9DPt1FlrzN5EKlIiTsOGp4f1i4kMcOSqhADLOy/EsQOwA7udh0+VO7US5/CqWZevSK9f2L9GljyeLc8R+bJ+bhNxHbtcJwaFYVUsWuSZpSo3LFWkUj5Ad9qLRXS3stl8MRX6/Mt86jHMKlj1LJ9GfEp/rDW726RRyW4uFWI/Z/GFDqAzBdQbcZ6oDgZObGiaUXFScsPv1Xp0RNGcuZRlFLbt0NIq8TBGFAEagEWoBvIKAaRbUA6UUA4jOaAVTagFcUBEQX0N5JdWjxKyQmNHV99ZYFidPYAgAHrkHptXKallESnGbcfIkeF8Oht08OGJY0yThRjJPUnuT7mvUkuh3GKjsiA+kXl0XFuZk8txbjxI3XZvJ5iufyyPQge9cWwU4tMhvhlc8eqMtu7WO4ntpmOhbxW16cDTcRkpLgHYBm0n+MmsSFs6KrK1u4dM/0vdfh+hHZXC22u3tLr70WheQ7f0mO/4l/Fnt7bfL3rCfGLPQ+2XGNQlhYXwKrzfwOK1MYRnLOCSr6TgDG50+pOP4TW5wzV2ahPmSwvI+Q+kFnjXK1/aa7ehYeC8k2lxBFOJJgHXzKHjbDjZgG0bbjOCD1qLiPFpaOfL4eV2edmQ6bhtV8FNSHHOHL9rBYN4cOGV0Kv1fUzBTrfqVwTt0zioOFcUu1d8lLCjjoSa/R1U0eyt89TObS1eVgkSM7HOFRSxOBk7CvoW0llmFGEpPEVkluActTXFwImjeNVOZWZSpRR1HmGzHoB756A1X1Oqr09Ttk9vzLOn0k7bOTGBnx7h31e4lhzkI2xPUqQGQn3KkVJTYrYKcejWSG+rwrHDyGFSkR1AdQDjhtkZpo4VODI6rn0BPmb8hk/lUV9qqrlY+yJ9NV4tsYGm8hcJju7qS6ZQYLYiC1j6qAg2bHrjDfN/2RXmgoddeZfae797/AG6Gttda392OyRpTgEEMAQdiD0IO2DmtAsPBER8KtbGOaWCCOLyM7kDSDoUsAT2X2Fc4S6HHLGCbQ64PxFbmCOdQQJEDaT1GeoPyORXqeVk9rmpxUkOa9OxN6ASegG70A1ioBzHtQDhRQCyHNAQPNnMwsTblkzHJKRI2M6UCn+eSD7hGFcylgr33eHjPcplhIOHcRuVg13BMaJFEDliXCOutuyRqMavRh74jXsyZTi/Ctly7j/gfL/EpLxbq6d1EjqX8CZBpCZZY3AP91ny4Unrv3I9Secs7rqtlPnm/wNMK7YPQ7frUpovc88XK4sFH/LvpUX90wIx/mKx0sayS84L82jMs/wBmvSTJmw56lSJFMQcquCzP5mx3OVO9ZOo4PXK2Tzj0PsuD6FarRxtc/l5Fu5osYntfFlt1laJdeC7xnGMuNab49um3aszhuvsqu8BP2W3jbO5T/wDH06q2MLPdkhOD83zOpS24cpSJclY5NIVTnoNO5O/TJNWtdoIzmp33bvpsbU+D6fSpRdmM9Nib4HzDb3ytE0YDY80UgDBgCDkZ2YA49wcVn3aXUcOn4tctvNfqU9fw2VSxZvF9yN47ze9nKbdbVNCgFNLlRpPTyBcA7dBV2nSz1tXiTulv1XYs6Hg9VtPPF4x6DqTmS8R4hPY+Gjuia/E1hdZA3C9Oveq/1Gm2ElC1yaT29xytFp5Rk67MtLpjGRl9KHApXEc6ouI0bxDqUOAcFTg41AbjbJycYre4Z/laI1WzXM91HO6T6HzNmh+vaqEVsm8NozQrWpCzmeCbjH0Zhw/TO5WN7pdPM4CvZ2OJDwTgFXEqpS8Rxae6wdiuuZ4yU6+G6afEPqniPGcc2F1/YmeTxi6Vu6RzuPmIJAP61S17b0/vcf8A5I7jpatNrpVVycuVPd7bmr/RLCF4ZER95pSfmJCn9FFa8OhxpP8ATyLfSBwOa6iRYGk15ZCgkCRMrruZgfiA0gDAJ835hOOUe6muU17PUqHFOI3sNhLZXi/cCxXKMJFOlg3hSkHYkKVBOOoyOprhtqOGVJznGtwn+JI8A4rHYrY2sZMr3eiaZuulJI9C6fQLoX5LE3rXqeMIkrsjUowW+TQ2qU0BI0Ak4oBCSgGkYoB1HQC6bUAtsN84x1PbHvQ8fQyjnHjEN9ZeK8XhXVvMkTqSOjh8gHupKE46gofmYZvmRl6i2NteejTK9ZyWT5N39aLzZYyR6Ai76jhGyZFBG5z22G2a4WO5BHk+/nL7mq/R1wO3gt/Ft5TJ4v8AmaZIg6qxC/ZOdj21DrU0IpLY09NXGMcosnFb5YYJZm6RozH+EZA+ZO1dN4RNOXLFtnn/AIiStnaofikae4b5MRFG35qjGsin29TZPyxH9X+Zn6n2NNCD6vLGlhDqaNcfE6ge+XA29fSvNU+XL9D7T6M244dJeTl+WTYuZ/8AB3H/ANmT/wBBr4TQvOqg/U50f+vD3r8yofRT8Vz8of6yVtcf+zD4mxx37nx/QiuLt9W4qWTYCVGwPSQKXH56m/Wr2n/zGhSn3T+RZoXj8PxLyfy6B/pG/wAd/wDjj/qa44P/ALT4s84R/tXnzZbrLh9x9YdLuQTJLDgFV0AGNwcYHQ+fIPt7VlW6ipUqenXK4y/MxbbaXWnSuVp+eeqA+kXiS+DHC7BRNKgZt8iJCGc7bncrVzh2ot1EpzlFZisJ439x7wrTvxJWRz7Kf4vZGe30MAvCA2q3aUHKED7NyCdJxtgEj8q2dPZNVKU17WN16m1fp3q+Huq1btfNB+Z+XzZuFMgcOGZSAQQobC6s9yMHb3ruvWV6lZr7GT9EtJLT12p92hrxPgdxbqjyx4STBVgQy7jUASDscb4PofQ1Zrvrti1CSeOvofHeFZVxSM5LrZ+opyrOEvICx8pcoflKjRf1cVFrIOWlljqln8Hk61TVfF7E+8n8zVPoiucWsls3x20zow9ic5/3Bx+VaVFinBSXff8AEj0/s81b6psuN9aLLG0b50OpVsMVOCMHDKQR+VStZLEkmsMxLivDuGW0ksZa4mfUyaIl8BIt9lLS6mdtsg7g9agcYoyJwqg2nliHKiQG+jW4JMMCSsBIuk6Y1eYK6ZPQljj29KR67nFKXirm6I1jlTj/ANdjeXwzGokKxgkFmjCrh2x0JbWPTy98ZqaMsmrRb4iyTLV0TCLUAhIKAZxUA7QUAulARPNfHjZRJKYw6GVUcFgraWVj5AdmbIG3pn5jmUsLJBfb4azgy7iws0u0aAGS0lEc/ggHzSIZEWAAjIGrbHYP32qF4T26GZZyKxOO6e5eOWuL3t7MBPw2ARROCGcFHgIGQVV8sWxjGFUe9SRbfYuUznY/aisF+Xb5VIXjOvpH4r9YlHDonCqPtbqXI0xRJhtz7bN89A9ap6zUKmGer6Jeb7IrWLxp+Eui3fuK9yrZpfXkk7Rj6vAqJFGw20gFYVKkbgBWcg9yBuKxtZc9Bo+vty7/APJ7timC1epcn9mJK83R5v7LPdlHQ/dkBA27b/lWPob5z0tspvL36v0PsdFiOmtSJ3myULZz6jjMTqPdmBCge5JrJ4dBy1MMdmV9Es3w96KDyJxqK1Mxm1DWqacKWyVL5Ht8Q619HxXST1KgoNbG9xXTy1HJ4eNs90KcMsJb++Nw0bJF4iuSwIGlcaUBPxEhQDj1NLr6tFpfDTy8YX7nN19ek0ngp5ljG3r1Yh9IUwa9YrvpVAcfiGSR/MV1wmLjpUpd8nfCUlpcPbOSy8W53jaPRahmlkwillKhGbYEk9Tk9s1mafhElZz2tcq369TLp4bJS5rWlFb9VvgT4dNDLxBYwytHb2/hxg7h3GA5XPXbP+2pL1ZVo5T6SlLL9F2PbIyr0nN3lLL9F2ID6QuGwxTr4ICl1JdF6Kc7ED7ud9var3CNRZbT/idujNPhGpnOtqx7Lo3/ADsBzTI1xb21wMsFjMUpH3JFx8Xpnr+nrXeiUabbKn3eV6pnnD+Wi6ypvq8r1RMtzTBPZfV1jaSZohGItBYawAA4PTAIDA9RiqlWjv02rdyklHfPu8jOs4VJX89mORPOc+ufxK/xjkm5giaYsjKqqzaGIdemTgjsT1BPTNbek4lptQ+SEt/I+K4tTbLU2apdG8oluC8weBPHxAf3U+IbxR/lzAD7TA7MAHH8Q6tUuil4E3p5dt4+sf7dDic08aiPR7S95ssUoYBlIKkAgjcEHcEHuK1y2nncg+bVmVFmt7SGeWMkr4pAMe27pnGem41KcVzLPYguylzRWWZNzXx6W5VJJ7bwbgB1V1R0WeF1KMoD5ORq23Ozt07wyeTMutlPDksMunLHGLaCeLh1mofdjcTMwGp1jOrRnd2yoG2wA27kdxaWyLlNkIyVcPiXlxUpeEnoBu9ANIqAcx7UA5UUBXubob0qDb3cUSsyJokjUlndgoBkYMMHI201xJPsVdQrH9llCMzcPvmluLZBJHAWREOIpJWYRiSMY8owWJAGxVsY2Aj+y8so5dNnNJb4NO5N4ncXUAnnWJQ5+zEWo+QbEuSTvkHYdMVLFtrLNKicpx5pDjmvji2drJOcZUYQHvI2yj5dz7A0k8LJ7dZ4cGzE+LStDD4TMTPckT3TH4gp80UJ99y7D1IHSsir/MXO5/ZjtH393+iKeol9Xp8P70t3+wbk3mL6pKdYJikADgYyCD5XGe4y23fPyrrX6KGrqdcvg/JlfQ6t6eeezNR4PxuG5jaSMkIshTU+FyQqNkDOw8+N/SvkdXwa6jljDMm85wtj6OjW13JvokMrvmy3W4it0PivJIqExlSsZZgvmboTvnA6Y3xU+m4BZKtztfK/L9yKziVcbFCO+Ser59zl5mkdXm7YOrpqyK3yjxNHV5zy8z06jcu54dXik10Z6dinM+uQFWMDoAPkAK9dkpdWettkRzjdrHZTliBrjaNQfvM40gKO5GSfkpNbPAaJz1Smlss5Zn8RsjCiSfcyjgV8kblJd4Jl8OYeik5WQejI3mB+frX2OrqlOHND7Ud1+3x6Hz+ivUJck/sy2Zqn0ZcSdfF4dO2ZLY5jP44TgjHsNSkezgdqt6S+N1anHv8AzHwL9Oa5OqXbp7i8vnBxgHBwTuAe2QOoqyWGY3zfzTJPFPaXkUSzQsrI8RYrrV1V1AYk/CzH8jtUEpZ2ZlX3uScJrdEzwLhPEY3iVJbe0R0LLCUWWRtAXV4vlBZzqBJ1j8ug6SkTV12prGEaEuQACcnG5xjJ7nHapTQQm4oBCSgGkYoB1GaAXSgILn6EPZshZFLOgUyP4YDg6lxJ0U+XYnY9Ns5rifQr6lZhgox4hLFc2v8Aayf4ZZXVzpkM6FR4S5UlXYOuQSe+/qY87+0UOdqcfG7fMunJPN8l/LJiFI4IxsdTNJqJGkE404xnPpt161JCXMXaL3a3thEV9Ilyst7b27n7GCN7uf0KoCQD/tx8papcRtlCrEOr2Xvex3hTvSfSO7MrvbtppHlf45GLN7E9h7AYA9gK6qrjVBQj0Ri32u2xzYjUhEBigyOeHPpljb8MiH9GBod1vE0/U9FcJgQpkqCc99+w9a6+qUf0L8EfS88vMeGRFYJsGYEgAdQuMn8sj9RXcaq49IpfAe01kC5uo12dlHsSM4yATj08w/WvZ1wksSSYUZdUFheHGVKYO+QV9M/03qKrSaer7EEvgey5++RK44hErqjEeZQQeoOplVQNt8lqklVXLaUV+B7GE8ZQvPaKVICgEjY4Gx7VXlw/TSTXhr8EeKyXmQamvzPVUOi6Vb7M0oyysjLjXFY7aJpZDsNlXOGduyL7+/YZNWuG8Onq7Mfd7sg1WphRDmZjnHeNS3UniSnpkKg+FF9FH9T1Nff0UV0QUK1hI+S1GondLmkR1TEBcOB8VMZsbzO8Mv1Sc/iiIzGx9cIzjJ6lBVLT/wCDqZV9pe0vykvx3NnxOeqF3dbM2HmG6lit5JIVVpEXUA+rSQCNXw79M498VrPoWbJNRbiZXxvnSC9hid4Atxb3EThB5hKmTrRWxkZwuVPtjNQuSfvM2zURsim1un+JNcrR3f8AaHi37IJnjYRxmQF0UjUQkKE6FwPiY+25Oa6jnm3J6FPxOazqaC1Sl8RagEJBQDSKgHSCgHCUAx5i4SLm3eA6fNjSW1YVlIIJ0kHt6/r0rySysEdtfiRcTLILG7tbhEmtZLgWYkZAkbyxkyLqjGoLsmrzb7jeoUmmZka7IyXMs4LryhfcUupg9wyxQoclEjTEmVICatRKkEgke1dx5n1LdLulL29kVfm1z9a4s3dYreMeyOYQ+PbAB/Os3W731L/k/lF4Ol/6z9Cg1cMM6gOoDs439KHq6npDgko0Ek4GAcnoBvk/yq0fSR3SErnwrlgFmAwkisNJDFZMLqUnHRl6gHcYrl4ZYjzVrdeXyGwsbNCM3Az5QBrQbDTpACj9hP8AaK8wjvntlukIy2tkCpEzb4GQ64CrEw1EtuBoJ3G/mHqKYR6pW46Bp7yyRlVoySn2YOgnAU5AOcex6elMoKFzWz6j1eZoT2YDGSSFH+WZPXfYfkSBXvMiN6eYjMwLal+FwHGxHXqCD0Oc7V8T9JtLy2xuX3tn70T6dtZi+xmf0rSDxoFzuImJHoGfY/npP6Vp8Ai1o1ldWzD4zJeKl6FGrbMc6gJbh5zZXw7D6mw/eM5X+jN+lUb/APc0v/t+RpaZ50ti9xtPFhdtZRSWshSZURyuhX8QGPePDbA5IOfatZ5xsXZ87gnDqZ1xril9cxgXFkWmgkSRJYYmcaRnWkjJqA7HqPh3FRvma3RRs8WS9qO6LN9HvLM0bm8uSpkmUsM6jIPEwxLHOkEjtgkZ6jcV1CON2WNLRKL559WXh6kLok1AN3oBpFQDmOgHKigCyzNkRxgGQjO+6ovTW/t6Dqx22AJAGZ86c6sr/V7GZlWNsyTggvNL3w3TSO56HAAwoGal97T5Yn0/CODRth4t62fRfqF5a5+kMireOuTgLdaQrKeyzquA8Rzv009f2lVanLxIcT4Cq4uzT9F1X7DjnGFVvhJINEV7C1tMevhTAKoYn2xCwJxlST0qDiNcnBWQ6xaa+HVfFHyG0LsS6TWPiZ1dWzxu0cgw6MVYehH/AGPUHuCDUtdkbIKcejMW6qVU3GQlXZGdQAGvAj0Jy0BJAAScPEu467gbj9atLofSVy9lMbcQs44mj8V2Yq8jKVCIqeLKZ/tC5IIJj0jA7dM4rlrBfhOUk+VfxLGwvHBaROYyCMeG5JZzh8PoAC7DCxnpgDC17scOVslkOI7IlQE3K5QBZMFSRtgDcEFe3w47U9k5zb5hYONQjHhW7OoCZZQCcFfL6sTgfewcAHuK8ydOmX3pD3hFx42svHjDDSrIQAuAPKWUZGpW39h0r1bkdseTGGH4vCFRWAACbegCnb8gMCs/iuj+taZ1rr1RzXZyyyzz9zHxH6xcyy6sqztoPT7NTpjGO3lArumpVVxguyPnNTa7bHJkdUpAdQFn4fwtmjgswCJb2VJXHdLZARHqHv55PktUav8AH1TmvswWF6t/a/Doa/huqiNX3pvL9xpPNvMsVtECyhwcrBBnAl07GWT/AEh2HQ9d8jTqWWKtbm5oNBZq7OSHTu/Izo8/8QMqymfZWyIgoWIjuhUblfmSR67VS+szzk+sf0e0vhOKzzef9uhqPAOORTQieI/Yk4kQ/FaydSG/0985+7kH4T5L8JKSyj43U6eensdc+qJxq6IBJ6AQkoBnHQDqOgDySFQMLqJIVRnGSTgZPYdyd8AHY0BF80cXksLdXVEaWafSS2oqMq7gbYLYVAg6evqKjsnyRyW9FpfrNvh5wVXiPA4OJ27XNpEIbuP+9hXAWQ4zjsCWwSr4Gejbg6YpQjbHK6mrptZdw2/wrXmH6eaM0qg9mfbRkpLK6M0zlGFOI2LWs58yjQHOCVaMaoHA7nQ7ofVYcVo0y54bn5zx3QRq1Eoro916fxlZ4jw53f6pc4jvYgFikY+S6j+4jOfvfhc9eh3rMnGWkm5RWYPqv6X5r080YTitQvDs2muj8ysTRMjMjqVZThlYYKn0Iq9CcZpSi8pmTZXKuXLJbhK6ODq8BvXIcubeA+sKf+hf/wCVZj0PoaHmuPuLFeWKyadWQVIIKnDbds9cdP0r1rJPGbj0EBwS38xMQbV8WvL5776s05UdeNPsxT+yoM58CPI6EIoI3zsQMjfemEeeLPzY4yiDGyj02H6CvTlvIUXafiFDwCd0ZSpdcEEdRQHnXmfh31e5ljxhdWpfTS24x8tx+VVpLDMDUV8ljRF14QJE1YWCwqtxdJnVvBbffnbs7jqsIP5t06bNQtuldJ00v/tLsvRecvyNSjTxoXi3dey8zTOU+XJYkku7kn63dEJ6GFJCq7DswGDjsEA7GtLT0RpgoxWyLdUJSk7J9X8jMeZuK/WbmSUfBnREB0WFPLGAOwx5serGqV0+aR+l8J0i0+miu73fx/YkuRuVjeysXJW3iwZXzgnuEU9jjcnsPmK7oq53l9Ctxnin1WPh1/bfyXmWfg/OyC9itLK1iS0aQR7KRJJnIMuc4Azv5gSQMk5OBZjcufkitjBu4VYtI9VdJ822z9fMv0aGMiIgaDq8Ngc7A5EZBG2AcDBOQh6Y3sGEKNQDeSgGkVAOkFAHlQkDSQGUhlz0yOx9iMg+xNAMObgtzYzARu0iAMI1XVKkq7oCq7kE7ZGQVJxkVzNZi0WNLa6rozXZlC5Av/C4hGhOPFDRup2I8pdCVO+dSYGfxGqtGYywz6HjHhXadWQaeH+ZXOdrURX90ijA8Utj08RRJ/76gvWLGbPBbHPRQz2yvwZL/Rbe+HdMvZlV/wA0bQSflHNMf4al0kt2jN+k1Oa4WeTx+JqnNPLEF9FomXDDOiQY1IT1x6qe6nY/PBq7KOT4m2mNi3Ms5i4TLBiLiKM6DyxX8Q1Oo+6soPxr+y3mG+kt1rJnpLKZOen+Mez939L+RDOSa5NQvdJfqVriHBZI18VSssHaeI6o/wCMdYz6hunTJrqrVwnLkfsy/pfX4efwKV2hnBc0faj5ojhVspGtcpcwQQWkOueNWCYwzrnbK/DnPap4ySW5t0WwjUsseXX0k2y7CYt+4jH+ZXH86c6PZaypdxrw7nQ3cnh29u7EDJaVgiIvqxGogegxk+lQX6yuiDnN4SPadR40uWuJZRq/F+gr5ez6UyU3yQ27Ze5qLS7bsqfP3MM9qYfCZftPE1agSfLox3/aP6Vs8M4lPWVOco43wZnELJaeSUe5Tm54vT0kUfJB/wB81o+IzN+vWiD84Xx/+pP5JGP/AG055HH1u7zGo+tX0nRpnUYLYUBF6+d9lUbk7moLtRCpc1jwdQqu1L8/UkuF2CJII4EF7d9QEBNrCR94k48Yg9zhB8xVTF+q23hD/wBz/wDqvmXYQq07xH25/JGmco8keC/1q8k8e6bfUd1jP7PqffAA6ADvp00QqioxWEiaFUnLxLHmX5Dzn2+8K2kYHBWKRlPo7j6vH/OfP8NSTliLZo6Orxb4Q82jBgKyT9OSNJtyYOXT4a+e5dlOkZLCSYoem5PhJprQinGnY+HvlC7iv+I8RT7+g1+i7gkqXX1ie1lWNI30SOojRGI3ciQhj5dQBUH4jn1HOmraeWixx7iFV0I11Sys74+Rpxl8Vw4/u0B0E/fY7FwPwgZAPfUT00k2z5gM9AIPQDOKgHUdAOFoAZLdXwWG46MCVZc9dLqQR+RoAjRMrxkyMw16cOEbAKtjDadWcgb5NAYv9JB/+Z3X70X/AOvEKzdR/qM+94Av8lH3v8yH4RfmCeOYKG0Nup6OjApIn5qzD864rnySyXuIaX6zRKvv295vHB+MrJEjJqmVx9k6jOtehEhO0ci9G1YyRtudI1E01lH5vZXKuThNYaHk0xYMskAdTs4RllAB6h0YAn5ANXpG1lblOveQ0LNPwu5NvL0ZMnwyfwup3Xr8LBgPwiq12lrujyzWf58vgV/BlW81PHp2KVxzhpiOOIWDQsT/AIm0A8NiT1eM+Q56nBDH0qg9NqKP9KWV5S3/AAfX3ZI7JVz2vhh+aIh+XnYF7aRLpBufCyJVH7du3nH5aqLXRi+W5OD9enwa2/Irz4e2uamSkvmRDDBIOxGxB2IPoR2NXU8rKKEoOLxJYNS5G4zYhFt4SY5DufF0hpX6ZDA4J9F7A7Z3r5zjmh1N/t1vMV939fU3+G6miK5Fs/zLTf3aQxvLIcKilj6nHQD3JwB7mvmtDo5am5Vr4+iNa+6NUHNmK8e41LdSmSU+yqPhjXrpX/ue9fodFEKYKutYSPj7753z5pHWHBJ5V8QIEi7zTMIoQPXW3xD90Go7dZVW+XOZeS3f895LVobbFl7LzY/srG21hIo5b+b8MatFAN+5H2jgeuymo19au2WIL8Zfsi1GvTVPCzOXyLxw3kS7uFUX0qwQDpaWwVFA9Gx5f11/MVZo0Fdb5nvLze7/AB7fAsNW2rE3yryRceD29vbpos7fyd3XAU7fE0rn7T5jVirySRLCuMFiKHhv2GC0RIPwmI+KpPZc4BBPqRp9SK9OzMPpR45n/hQwLlle4KnKppB8K3B741FyfX0zgVNTZtyo+m+j2hlKz6xJbLp7zPaon2JuPJDt/Z1kqsV1awSApOkeK22oEdQvatWr7CPzXiP+7s/7P8yfNkp3ctIRv5zkAjoQgwgI9QM1IUhVqARegEJKAZxUA6joBwlALLQCV8fKp9JYf5yoD/WgMW+kqMjidzn7xiI+XgRj+oNZ2p2mfefR+WdGl5NlbijZmCqpZmICqBksxOAAO5qCKcnhGvbbCqDnN4SLLNxJ7CGS0hmPjSkG5dGOiIgY8GIjYyY2eQemkdPLZlPwo8qe589Tpf8AyN/1m2OILou79WJDlO8hgS7jBUlS5ETFJ40J2dguG0nGcjp3rnw7IrmRZ+u6C616ecVhbJtbN+S8iV4B9IMisou8yAbCePC3CD9r7sq/ssPfzGpK9T2kUdf9Hlhz0/4P9H+5qNjxiOWLWWSWFgR4yDKe4lQ7xkZ3zkDBzp6VcTTR8pZXKDcJrD8mQnGPo5s5iJIM28nVXhOFyehCDYfwla4nVGSw0VJaaOcw9l+hUeYOX72Ef8ZbLexAbTxeW5jX94DUR3wwYerVmy4e63zaeXL6dY/h2+B5OcsYvjzLzXUqU/Bg6GW0kM6KMspGm4hHq8Y+Jf20yPbbNeQ1bjJQvXK+z+6/c/0ZVnolJc9DyvLuiV4pxCfiR1AiK1ixl5H+zU4ALO/+bKewGTuOmSTzKVGlbUF7Ut8Lq/2R2426veTxFeYpwLhniNjh9obhgd7u5XESn1jiPlX1GrLD0r1afUX/AOrLlX9Mf1l+xJW669qI8z/qfQuth9G3iMJOI3T3D/gVisa+oB2IH7oT5Vfp0ldKxBY938yyR0zs3uln07FttooLYeDbwqD18OJQMZ+9I3Re+7bnBxk7VYSSJ4wjFYSKVzbz1HFmNCtxKMgqCTaxH0cjBnYHtsNvukVDZeobdza4fwa3Ve1L2Y+fn7iifWr3ic6xPOXZydKO4jhGkasBB5cgexPTrVTnnY8ZPpvqmi4fU7HHOO+Mv+wblnj83DpmXS2gkrNDnSdtiV/DIOx79Pce1Wut4Z5xDhtWuqVlWObqn5+8b8w8H8ErLE5ltp8tDN3J3LRy+kqnOQeuCeuQPLq8e0ujOuFa5WR8Ca5Zx2x+xEVAbBuXIyEWdip/5Msn6uuP5SVrVfYR+acQedVY/wDk/wAyztXZTE2oBFqAQkoBnFQDpKAcJQCy0AW7hLoyrsxB0k9A3VT+RANAZP8ASxalrmG4VToniVRsSfFRjlCB9/DKNPXKn0qlqYNtNH1n0d1dcKpwm8Y3+HcHhXL/AIEZC3lpFxB/KVmnCPbRsu6xhQftmB3b7oOF7k9118i2e5W1vEXqrVKUJOpeXf3kDfcAuLCWN7q1DxK6nIbVDIAQdJcfDn0YAn0IqvKqUHzSWTbp4jp9ZU6qpckmsL09xbOFcUTiDM93PCp8RlgtwfCmdWUgW7zbfYttnrlu4+EzQmrPtP4GRqtLPRJRpi3t7Uuq/wCyXZr5fMR5p5WE1zK6GC1hhSKOR28kTTlQxCgD0dAT8upzXltXNLK2RNw/ibooUZc05SbaS6pfzJVkmvOGXLqrmORCoYA6o5ARqXIOzqR07jfGDUKlOqWDVnTpeJ0qeOvR91/+Gg8sc+QSkKxW1mJ3Vv8ACysfwn/KYnP5t981crvjP3nymu4NfpvaXtR81+pe4L4ZCuPDc9FYjDfuN0f123x1AqcyCsc28jpMfrFofAu18yunlVz6OOmT64375FQ3UQti4yWUyvOnD569pEJy5yhPeMJ+JLojjJEdqq+GuoEh3ZQdsnJ65bPXTgGvpdFXQnhde73f4+R5iy95t2S7fqaGWigRV8qKNkRRjp0VEXqfYCrxYSSWEQfMPMsVuuZ5DCCPLGuGuZR7KDiNffrv1Q1zOcYrLLOm0luoly1RyZdx3nC5u/sIEMULEgQxZaSUnr4jjzOT3A65OdXWqM9RKe0T63R8Eo00fF1DTa8+i/cHl/lYfXYra9GnXGziOORdeQCQj4+E4UnHy3615Cn21GZNq+J/5WVum7NLLXbzRarvgix2iRyTmKON1ls7xo2V4tR1GKdMAq3oTgHboQAZ3BKOG8Y6MxIaydmocoR5nJYlHs/VfzYp3Nlyt7fE2kbSF1RfKhDSsow0mjqo6bnGAMnFV7P8Sfso3dB/kNJjUyxu9vL0J7gfLc1qrR381rHazj7WCa4CyBseWSPAKiRcDcNvjHYEWK63FYm9jE1+vr1Ninpoy513S6lT47wJ4JFWNhNHMf8Ah5Y8FZstpCgjbWCQCPX2NV50tS23TN3R8WhbTJ2+zKK3RuHA7Pw1SMHIt4YoAf2lUeIR6j+7HzUjtWilhYPg7JucnJ92SZr04E2oBJqAbvQDOKgHSUA4SgFkoBRaAZXtgxDeEwGrJwQPJIQQJYmKsEffO6kE9gSxI9TwUWD6LVL5klkYE5OtkGSTkksupnz1PwE+oqv9WjnLNp8e1CrVcFGK9EXa14GIYRDFpMYUqYZBmJlP3cYJT0z5u5YMTmp8djGcm5cz6lF4/wAgAv4lgTDMp1/VnbGSpB1QSZx6HGSBkDKdKr2adPeOzN7RccnCPhahc0Xt64/Ugv7fQxta8VgnZ0maXyERuXYEESKcDG5ww7HbbrA57ctqNWGk5pq/h84pNYw+389Seh4TJPaWkCa2jucT3V0516EjA0xGQ9CoAXtuvT4sScjlGK7PdsoPURp1FtssKUPZjFbZb74+ZVea+BQwotxE+I53fwYW3bwFG0urPwk7gHfDr71DbXGKTRscN19t8nVYsuK3l6+X89QvDOZb6y+xOTHj/D3KFk09tIbDKPTB0+xr2N04bM41HCdHrMzreH5x6fFFpsPpLhA89tPEen2EqyIB7Ry4VR7AVPHVR7mNb9G9RF+xJP5Dmb6TLXHS9f2It4h/vjIYflXT1MEQx+j2sfXC+JXuJfSLO2RbRJb5GDJnxpyPeVx/UH51DPVP7pq6b6NwjvdLPotkQnDOEy3XizvISsZRp5GbXKEY4L6ScvpAJOT0X8qijGVjy2ad+op0KjVXHd9F2z6v1L5b8It7SZxbRyLcW0IuElaTVHdwYxKpA8q5yVGwwcHfFWVCMHt1XzPnbdXfqa14sk4yeMJbxfb1GnHeYbKJ0ngl1lSJoYIkWNI3cAyNcSblnbzAjqAeneuZ2wWGv57ybScP1NkZVzXKukpPfKXRRX895E8M5au79muLmQwQO/iF3LHUWOwhjY56YVSe2MaulcxpnN5kT6jimm0UVXpkpSSxn+/c0vgHAY4E0W0fgocapGGbiX3OoeQfvDbJGlauRgorCPl9Rqbb589jyyI5j+j2K4kMoLBz1cNhyAMAOGBEmBgZ8rbbs1cWVRn1LWi4nfpNoYx5MV5V5RktAyidijHUNYjPhvgqXiQAhXKkjVqx6qa9rr5FhMj1utlq588opP07+8tkMIRQqjAHuSfUkk7kk5JJ3JNSFIFqATagEXoBCSgGcVAOo6AcJQCy0AqtAGFAHoARQCdxbq40uuR19wR0II3BHYjcUBAcxctx3CaZ4zKozpkXAuYR+y2PtF9jvtuHNcygpLDLGm1VunnzVvH6+8zLjfLV5aRsYZXntGJ1+GzhdviW4hU5HoT7HOnpVOdM4fZ6H1Ok4ppdU148VGfaWPyfb4jw8XsuIXFtJPmBkKq4Z1NsYkBYBSR5ctgY2GGO+1OeFkk5bNfgPqur0NNkasTjLo19rL/sWTjdhLeI1vM0ZlmutcAQq5t7VMapCR2IGMdy4HfaWcXNYfn8jL0t8dLNW1ppRjvnbmk+387IhL/kq2e7hiiMsUUlvJKdWS4MbYzpfcZyNv6VFKiPMkvIv08Z1EdPKc8SkpJem/uEeG8iQSlGW5kkilgkliCIkcrGJlV4z4h0g5dcE4HX0zSOmi987El3Hb4JxcEpJpPOWlnp0JDgNnaQSyxrbSeIESQHSl6yxjIdT4eVRyR0G+4xUkIwjJpLf8Snq79TfXGcprly13hv296IW5lHCuJZRjJE6AyR6VQ+HLktGUGACMAgbdhULfhW+hpVQlxLQYltKL2fqu/xEoeK316v1GzRvAXKgDGpYSxKJNN0Cqu2NsgY81eqU7PZj0OZ06PQy8e95s8l5+aX7ls5V5EhhIdgtzMD8R/wsLA/dHWVwc/mv3DViuiMPeYWv4zdqvZXsx8l395fILIA63Jd/wATdFz1CL0QdvUjGSanMgc0ABFABQBTQBGoBNqARagEJKAZxUA6SgHCUAstAKigDrQBhQBqAEUB1ANriyDHWpKP+NcZOOzA7OPmNsnGOtAUzmLkW3mJZozDIes1spZGPrJb7kZJ+7k+rVFOmMzS0fFtRpdovK8n0KceUOJWodrOQSxuNLPbSDLKOzITkHr8JNVvAsh9g3ocY0Oq5frEMNfFfz3oSi5vu4Hja6tyxjhkhHjLLE7LIUJLs+5byYzt1NeKyyOMo7lodDdGSptSy0+q2xnotvMTh56fx4Wht41SCOSKOBSzDEgGrJ6tkqp/hrxWzymo9DqXDNOqpRsu3k03JtdviLxWfFLtkaCy+rBAwVoYzZqA+nVlmIZx5R0zXvLbN7LHyI/rHDtNFqU3ZnHX2unyRM8H+juMtqupmuHz5o7cnRq7iS4bBz7Aq3zqWGmXWW5nanj9klyUR5F8/wBkaDYcGVEEelI4x0ghGmP31tsZO+c4BzuD1qyklsjBnOU3zSeWSiqAAAMAbADYAelenIJoDqALQAGgCmgCGgE2oBJ6AbvQDOKgHSUA4SgFkoBRaAUFAGFACKANQHCgOoDqAbzWMbnUVw3TUpKPj01qQcfnQBPqbjpPJjsCI2H5krqP5mgO+qSf88j9xIwf+oNQHf2ch/vC0nr4jEqfnGMJ/wBNAOlUDYDAHT29qANQHUAFAdQAUAWgCmgCGgE2oBJ6AbyUAzioB1HQDhKAWWgFVoA4oA1AGFAdQA0ANABQA0B1ABQA0B1ABQA0AFAdQAGgC0AU0ARqATagEmoBvJQDOKgHSUA4SgFloBUUAdaAMKANQA0BwoAaA6gAoAaA6gAoAaA6gOoADQHUAFAAaAIaAIaATagEnoBu9AM4qAdR0A4SgFkoBVaAOKAMKAEUAagOFAdQA0B1ABQA0B1AdQHUB1ABQHUAFAFNAAaAI1AJNQCT0A3koD//2Q==']

iniciaram_2018['logos'] =  logos

iniciaram_2018

,Nome da IES,Iniciaram em 2018,logos
0,FSG,4370,https://upload.wikimedia.org/wikipedia/commons...
1,UCS,3001,https://www.ucs.br/site/static/img/capa2016/lo...
2,FTEC,1859,https://www.ftec.com.br/static/front_end/_imag...
3,Anhanguera,410,https://i1.wp.com/tempofm.com.br/wp-content/up...
4,FTSG,111,https://qb-assets.querobolsa.com.br/logos/colo...
5,Murialdo,100,https://www.sinprocaxias.com.br/igc/uploadAr/F...
6,Fátima,100,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
7,IDEAU,78,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."


In [0]:
def gerar_texto(dados, fluxo, coluna):
  '''
  função para gerar a coluna com os textos
  representando o fluxo dos alunos
  
  parâmetros:
    dados -> pandas dataframe a ser usado
    fluxo -> pandas dataframe com os dados dos fluxos dos alunos
    coluna -> nome da coluna em que estão os nomes da IES
  '''
  ies = list(dados[coluna])
  string = []
  for cada in ies:
    if cada == 'UCS':
      string.append('')
    else:
      valor1 = fluxo.loc[(fluxo['De Onde']==cada) & (fluxo['Para Onde']=='UCS'),'Quantidade'].values
      valor2 = fluxo.loc[(fluxo['De Onde']=='UCS') & (fluxo['Para Onde']==cada),'Quantidade'].values
      
      if valor1.size == 0: valor1 = 0  
      else: valor1 = valor1[0]
      if valor2.size == 0: valor2 = 0  
      else: valor2 = valor2[0]
      
      texto = '{} -> UCS: {}'.format(cada, valor1) + '<br/>' + 'UCS -> {}: {}'.format(cada, valor2)
      string.append(texto)
      
  return pd.Series(string)

In [21]:
# adicionando a coluna de texto
iniciaram_2018['texto'] = gerar_texto(iniciaram_2018, fluxo_ies, 'Nome da IES')
iniciaram_2018

,Nome da IES,Iniciaram em 2018,logos,texto
0,FSG,4370,https://upload.wikimedia.org/wikipedia/commons...,FSG -> UCS: 118<br/>UCS -> FSG: 302
1,UCS,3001,https://www.ucs.br/site/static/img/capa2016/lo...,
2,FTEC,1859,https://www.ftec.com.br/static/front_end/_imag...,FTEC -> UCS: 15<br/>UCS -> FTEC: 41
3,Anhanguera,410,https://i1.wp.com/tempofm.com.br/wp-content/up...,Anhanguera -> UCS: 4<br/>UCS -> Anhanguera: 10
4,FTSG,111,https://qb-assets.querobolsa.com.br/logos/colo...,FTSG -> UCS: 0<br/>UCS -> FTSG: 3
5,Murialdo,100,https://www.sinprocaxias.com.br/igc/uploadAr/F...,Murialdo -> UCS: 8<br/>UCS -> Murialdo: 3
6,Fátima,100,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",Fátima -> UCS: 1<br/>UCS -> Fátima: 2
7,IDEAU,78,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",IDEAU -> UCS: 0<br/>UCS -> IDEAU: 3


In [0]:
# csv
iniciaram_2018.to_csv('quinta_visualizacao_nos.csv', index = False)